# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:

def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())
            
        self.model.add(dense(label_dmi, weight_init))
        
    def call(self, x, training=None, maks=None):
        x = self.model(x)
        return x

# Create model(function)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.9941, train_loss: 2.08932400, train_accuracy: 0.3750, test_Accuracy: 0.2047
Epoch: [ 0] [    1/  468] time: 1.0844, train_loss: 2.03221130, train_accuracy: 0.4219, test_Accuracy: 0.3578
Epoch: [ 0] [    2/  468] time: 1.1684, train_loss: 1.97564161, train_accuracy: 0.4531, test_Accuracy: 0.4239
Epoch: [ 0] [    3/  468] time: 1.2604, train_loss: 1.87699127, train_accuracy: 0.5469, test_Accuracy: 0.5090
Epoch: [ 0] [    4/  468] time: 1.3425, train_loss: 1.68688023, train_accuracy: 0.6328, test_Accuracy: 0.5781
Epoch: [ 0] [    5/  468] time: 1.4205, train_loss: 1.59199762, train_accuracy: 0.6641, test_Accuracy: 0.6406
Epoch: [ 0] [    6/  468] time: 1.5067, train_loss: 1.52409101, train_accuracy: 0.7109, test_Accuracy: 0.6796
Epoch: [ 0] [    7/  468] time: 1.5928, train_loss: 1.40514231, train_accuracy: 0.7344, test_Accuracy: 0.7206
Epoch: [ 0] [    8/  468] time: 1.6728,

Epoch: [ 0] [   74/  468] time: 7.0436, train_loss: 0.20806387, train_accuracy: 0.9531, test_Accuracy: 0.9180
Epoch: [ 0] [   75/  468] time: 7.1196, train_loss: 0.26502126, train_accuracy: 0.9219, test_Accuracy: 0.9199
Epoch: [ 0] [   76/  468] time: 7.2066, train_loss: 0.35210466, train_accuracy: 0.8906, test_Accuracy: 0.9231
Epoch: [ 0] [   77/  468] time: 7.2866, train_loss: 0.28671646, train_accuracy: 0.8984, test_Accuracy: 0.9229
Epoch: [ 0] [   78/  468] time: 7.3646, train_loss: 0.20067668, train_accuracy: 0.9375, test_Accuracy: 0.9206
Epoch: [ 0] [   79/  468] time: 7.4466, train_loss: 0.26827383, train_accuracy: 0.9219, test_Accuracy: 0.9182
Epoch: [ 0] [   80/  468] time: 7.5296, train_loss: 0.38603455, train_accuracy: 0.8906, test_Accuracy: 0.9219
Epoch: [ 0] [   81/  468] time: 7.6156, train_loss: 0.32823896, train_accuracy: 0.9062, test_Accuracy: 0.9245
Epoch: [ 0] [   82/  468] time: 7.6916, train_loss: 0.17744973, train_accuracy: 0.9688, test_Accuracy: 0.9253
Epoch: [ 0

Epoch: [ 0] [  151/  468] time: 13.4493, train_loss: 0.26496500, train_accuracy: 0.9375, test_Accuracy: 0.9355
Epoch: [ 0] [  152/  468] time: 13.5363, train_loss: 0.13555130, train_accuracy: 0.9766, test_Accuracy: 0.9342
Epoch: [ 0] [  153/  468] time: 13.6193, train_loss: 0.13707608, train_accuracy: 0.9688, test_Accuracy: 0.9357
Epoch: [ 0] [  154/  468] time: 13.7053, train_loss: 0.28140619, train_accuracy: 0.9141, test_Accuracy: 0.9370
Epoch: [ 0] [  155/  468] time: 13.7893, train_loss: 0.12601876, train_accuracy: 0.9688, test_Accuracy: 0.9359
Epoch: [ 0] [  156/  468] time: 13.8713, train_loss: 0.16330238, train_accuracy: 0.9531, test_Accuracy: 0.9353
Epoch: [ 0] [  157/  468] time: 13.9623, train_loss: 0.20351180, train_accuracy: 0.9453, test_Accuracy: 0.9334
Epoch: [ 0] [  158/  468] time: 14.0473, train_loss: 0.11610410, train_accuracy: 0.9688, test_Accuracy: 0.9321
Epoch: [ 0] [  159/  468] time: 14.1303, train_loss: 0.19989398, train_accuracy: 0.9375, test_Accuracy: 0.9296
E

Epoch: [ 0] [  226/  468] time: 19.7033, train_loss: 0.22628200, train_accuracy: 0.9531, test_Accuracy: 0.9405
Epoch: [ 0] [  227/  468] time: 19.7853, train_loss: 0.11899409, train_accuracy: 0.9531, test_Accuracy: 0.9413
Epoch: [ 0] [  228/  468] time: 19.8683, train_loss: 0.10189028, train_accuracy: 0.9844, test_Accuracy: 0.9427
Epoch: [ 0] [  229/  468] time: 19.9513, train_loss: 0.17581919, train_accuracy: 0.9609, test_Accuracy: 0.9443
Epoch: [ 0] [  230/  468] time: 20.0383, train_loss: 0.25136474, train_accuracy: 0.9453, test_Accuracy: 0.9460
Epoch: [ 0] [  231/  468] time: 20.1163, train_loss: 0.20803398, train_accuracy: 0.9688, test_Accuracy: 0.9486
Epoch: [ 0] [  232/  468] time: 20.2003, train_loss: 0.20886222, train_accuracy: 0.9297, test_Accuracy: 0.9503
Epoch: [ 0] [  233/  468] time: 20.2843, train_loss: 0.21575356, train_accuracy: 0.9609, test_Accuracy: 0.9503
Epoch: [ 0] [  234/  468] time: 20.3663, train_loss: 0.23908041, train_accuracy: 0.9297, test_Accuracy: 0.9505
E

Epoch: [ 0] [  301/  468] time: 25.7953, train_loss: 0.13254723, train_accuracy: 0.9688, test_Accuracy: 0.9504
Epoch: [ 0] [  302/  468] time: 25.8723, train_loss: 0.18112852, train_accuracy: 0.9453, test_Accuracy: 0.9504
Epoch: [ 0] [  303/  468] time: 25.9493, train_loss: 0.15335870, train_accuracy: 0.9531, test_Accuracy: 0.9523
Epoch: [ 0] [  304/  468] time: 26.0393, train_loss: 0.19834641, train_accuracy: 0.9219, test_Accuracy: 0.9530
Epoch: [ 0] [  305/  468] time: 26.1153, train_loss: 0.14205167, train_accuracy: 0.9688, test_Accuracy: 0.9540
Epoch: [ 0] [  306/  468] time: 26.1923, train_loss: 0.17483720, train_accuracy: 0.9453, test_Accuracy: 0.9560
Epoch: [ 0] [  307/  468] time: 26.2793, train_loss: 0.09812194, train_accuracy: 0.9688, test_Accuracy: 0.9561
Epoch: [ 0] [  308/  468] time: 26.3553, train_loss: 0.10570629, train_accuracy: 0.9609, test_Accuracy: 0.9547
Epoch: [ 0] [  309/  468] time: 26.4373, train_loss: 0.17925787, train_accuracy: 0.9531, test_Accuracy: 0.9542
E

Epoch: [ 0] [  376/  468] time: 32.0303, train_loss: 0.08363900, train_accuracy: 0.9688, test_Accuracy: 0.9529
Epoch: [ 0] [  377/  468] time: 32.1173, train_loss: 0.08038691, train_accuracy: 0.9688, test_Accuracy: 0.9500
Epoch: [ 0] [  378/  468] time: 32.2003, train_loss: 0.14176440, train_accuracy: 0.9531, test_Accuracy: 0.9504
Epoch: [ 0] [  379/  468] time: 32.2803, train_loss: 0.17750338, train_accuracy: 0.9609, test_Accuracy: 0.9490
Epoch: [ 0] [  380/  468] time: 32.3633, train_loss: 0.09278379, train_accuracy: 0.9766, test_Accuracy: 0.9498
Epoch: [ 0] [  381/  468] time: 32.4463, train_loss: 0.11049089, train_accuracy: 0.9844, test_Accuracy: 0.9498
Epoch: [ 0] [  382/  468] time: 32.5263, train_loss: 0.18713714, train_accuracy: 0.9141, test_Accuracy: 0.9509
Epoch: [ 0] [  383/  468] time: 32.6133, train_loss: 0.09964812, train_accuracy: 0.9609, test_Accuracy: 0.9538
Epoch: [ 0] [  384/  468] time: 32.6923, train_loss: 0.11087622, train_accuracy: 0.9609, test_Accuracy: 0.9550
E

Epoch: [ 0] [  451/  468] time: 38.1691, train_loss: 0.20312254, train_accuracy: 0.9375, test_Accuracy: 0.9628
Epoch: [ 0] [  452/  468] time: 38.2481, train_loss: 0.12366292, train_accuracy: 0.9531, test_Accuracy: 0.9640
Epoch: [ 0] [  453/  468] time: 38.3251, train_loss: 0.11296827, train_accuracy: 0.9766, test_Accuracy: 0.9641
Epoch: [ 0] [  454/  468] time: 38.4081, train_loss: 0.22318560, train_accuracy: 0.9375, test_Accuracy: 0.9633
Epoch: [ 0] [  455/  468] time: 38.4861, train_loss: 0.19034690, train_accuracy: 0.9297, test_Accuracy: 0.9621
Epoch: [ 0] [  456/  468] time: 38.5712, train_loss: 0.20190302, train_accuracy: 0.9297, test_Accuracy: 0.9609
Epoch: [ 0] [  457/  468] time: 38.6552, train_loss: 0.10578409, train_accuracy: 0.9688, test_Accuracy: 0.9606
Epoch: [ 0] [  458/  468] time: 38.7363, train_loss: 0.12944174, train_accuracy: 0.9453, test_Accuracy: 0.9609
Epoch: [ 0] [  459/  468] time: 38.8213, train_loss: 0.08394381, train_accuracy: 0.9766, test_Accuracy: 0.9612
E

# Test Accuracy : 96.46%